In [5]:
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import urllib.request
import pymysql 
import math

hdr = {'User-Agent': 'Mozilla/5.0'} 
url = 'https://www.melon.com/chart/index.htm'

driver = webdriver.Chrome('C:\chromedriver.exe')

# create cursor

conn = pymysql.connect(host = "127.0.0.1",user = 'root', passwd = 'root', db = 'gradproj')
cur = conn.cursor()

#create table
query1 = """drop table if exists comments"""
cur.execute(query1)

query2 = """create table comments(title varchar(100), album varchar(100), writerID varchar(50),comment varchar(1000));"""
cur.execute(query2)

#save songID 
req = urllib.request.Request(url, headers=hdr) 
html = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html, 'html.parser')

lst100=soup.select('.lst50, .lst100')
songID = []
title=[]
album=[]
for i in lst100:
    songID.append(i.attrs['data-song-no'])
    title.append(i.select_one('.ellipsis.rank01').a.text)
    album.append(i.select_one('.ellipsis.rank03').a.text)

baseUrl = 'https://www.melon.com/song/detail.htm?songId='
#노래100개
for i in range(0, len(songID)):
    plusUrl = songID[i]
    url = baseUrl + plusUrl
    driver.get(url)
    #마지막 페이지 찾기
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    result = bs.find(class_="fl_left").find('span').get_text()
    total_count = int(result.replace(',', ''))
    lastPage=math.ceil(total_count/10)
    #댓글 1페이지부터 마지막 페이지까지 크롤링
    for p in range(1, lastPage+1):
        lasturl = url + '#cmtpgn=&pageNo=' + str(p)
        driver.get(lasturl)
        driver.implicitly_wait(60)
        source = driver.page_source
        bs = BeautifulSoup(source, 'html.parser')
        
        replies = bs.find_all(class_="list_cmt")
        writerID=[]
        comment=[]
        for j in replies:
            writerID.append(j.find("a").attrs["data-member-key"])
            comment.append(j.find(class_="cmt_text").get_text())
        for k in range(0, len(writerID)):
            sql = "INSERT INTO comments VALUES (%s, %s, %s, %s)"
            values = (title[i], album[i], writerID[k], comment[k])
            cur.execute(sql, values)
            


    
conn.commit()
cur.close()
conn.close()
driver.close()
print("the end")

the end
